In [9]:
# !pip install darts
# !pip install -U kaleido

## **Step 1: Import all the libraries and modules**

In [16]:
import pandas as pd
import numpy as np
import os
import sys

%load_ext autoreload
%autoreload 2

target_dir = os.path.abspath(os.path.join(os.getcwd(), '..' ))
sys.path.append(target_dir)
          
from Models.XGBOOST.XGBBOOSTModel import xgboost_model
from Models.XGBOOST.xgb_evaluation import evaluation_of_model_xgb_Model
from Models.XGBOOST.xgb_evaluation import load_model_xgb_model
from Models.ARIMAModel.preprocessing import preprocess_df_to_ts

from sklearn.model_selection import  train_test_split
import logging
import gc

import sys
import warnings
warnings.filterwarnings("ignore")
logging.disable(logging.CRITICAL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **Step 2 : Reading Data Daily Traffic Accident Forecasting**

In [30]:
daily_traffic_accident_data_path = '../data/processed/daily_data.csv'
daily_accident_df = pd.read_csv(daily_traffic_accident_data_path)
daily_accident_df

,daily,daily_accident
0,2018-01-01,234
1,2018-01-02,408
2,2018-01-03,231
3,2018-01-04,225
4,2018-01-05,342
...,...,...
1821,2022-12-27,183
1822,2022-12-28,177
1823,2022-12-29,191
1824,2022-12-30,193


## **Step 3 : Split the Data into train and test data**

In [31]:
processed_train_df, processed_test_df = train_test_split(
        daily_accident_df, test_size=0.245, random_state=42, shuffle=False)

## **Step 4 : Preprocess the data into Time series object**

In [32]:
print('[INFO] Converting data into times series types')
ts_train = preprocess_df_to_ts(processed_train_df,'daily','daily_accident','1D')
ts_test = preprocess_df_to_ts(processed_test_df,'daily','daily_accident','1D')

[INFO] Converting data into times series types


## **Step 5 : Training the Model**

In [40]:
print('[INFO] Train XGBOOST Model...')
os.makedirs('../pkl',exist_ok=True)
trained_model_file_name = f'../pkl/XGBoost.pkl'
xgbmodel = xgboost_model(ts_train)

xgbmodel.save(trained_model_file_name)


[INFO] Train XGBOOST Model...


## **Step 6 : Loading the Trained Model**

In [41]:
xgbmodel = load_model_xgb_model(trained_model_file_name)

## **Step 7 : Evaluation of the XGB Model**

In [42]:
evaluation_sarima_df = evaluation_of_model_xgb_Model(trained_model_file_name,
                                                    ts_test=ts_test,
                                                    ts_train=ts_train)

gc.collect()

Evaluation of input window : 30 & Horizon : 418
metrics: {'MAE': 54.45, 'RMSE': 68.74, 'MAPE': 22.12, 'MSE': 4725.45}
Evaluation of input window : 60 & Horizon : 388
metrics: {'MAE': 55.23, 'RMSE': 68.77, 'MAPE': 22.55, 'MSE': 4729.33}
Evaluation of input window : 90 & Horizon : 358
metrics: {'MAE': 52.84, 'RMSE': 65.71, 'MAPE': 20.73, 'MSE': 4317.3}
Evaluation of input window : 120 & Horizon : 328
metrics: {'MAE': 53.43, 'RMSE': 66.13, 'MAPE': 21.13, 'MSE': 4373.56}
Evaluation of input window : 150 & Horizon : 298
metrics: {'MAE': 49.77, 'RMSE': 63.88, 'MAPE': 19.4, 'MSE': 4080.04}


0

## **Step 8 : Saving the Statistical Results**

In [43]:
print('[INFO] Generating Statistical Results of RNN-LSTM Model')
statistical_result_save_path = os.path.join("../StatisticalResults")
os.makedirs(statistical_result_save_path,exist_ok=True)

evaluation_sarima_df.to_csv(
f"{statistical_result_save_path}/xgb_evaluation_results.csv", index=False
)

[INFO] Generating Statistical Results of RNN-LSTM Model
